<a href="https://colab.research.google.com/github/bublaiSAURUS/Reinforcement-Learning/blob/main/ExperimentingwithGym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stable-baselines3[extra]

In [2]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/dtypes.py:35: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  from tensorflow.tsl.python.lib.core import pywrap_ml_dtypes


In [3]:
env_name = 'CartPole-v1'
environment = gym.make(env_name, render_mode = 'human')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
os.environ['SDL_VIDEODRIVER']='dummy'

In [5]:
episodes = 10
for episode in range(1, episodes+1):
  state = environment.reset()
  done = False
  score = 0
  while not done:
    environment.render()
    action = environment.action_space.sample()
    n_state, reward, done, info = environment.step(action)
    score+= reward
  print('Episode:{} Score:{}'.format(episode, score))
environment.close()

Episode:1 Score:18.0
Episode:2 Score:19.0
Episode:3 Score:22.0
Episode:4 Score:12.0
Episode:5 Score:25.0
Episode:6 Score:9.0
Episode:7 Score:12.0
Episode:8 Score:17.0
Episode:9 Score:21.0
Episode:10 Score:30.0


In [6]:
environment.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [23]:
import time

import gymnasium as gym
import numpy as np

from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

In [24]:
from typing import Callable


def make_env(env_id: str, rank: int, seed: int = 0) -> Callable:
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environment you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    :return: (Callable)
    """

    def _init() -> gym.Env:
        env = gym.make(env_id)
        env.reset(seed=seed + rank)
        return env

    set_random_seed(seed)
    return _init

In [25]:
env_id = "CartPole-v1"
num_cpu = 4  # Number of processes to use
# Create the vectorized environment
env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

model = A2C("MlpPolicy", env, verbose=0)

In [26]:
# By default, we use a DummyVecEnv as it is usually faster (cf doc)
vec_env = make_vec_env(env_id, n_envs=num_cpu)

model = A2C("MlpPolicy", vec_env, verbose=0)

In [27]:
# We create a separate environment for evaluation
eval_env = gym.make(env_id)

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward:.2f}")

Mean reward: 9.6 +/- 0.49


In [28]:
n_timesteps = 25000

# Multiprocessed RL Training
start_time = time.time()
model.learn(n_timesteps)
total_time_multi = time.time() - start_time

print(
    f"Took {total_time_multi:.2f}s for multiprocessed version - {n_timesteps / total_time_multi:.2f} FPS"
)

# Single Process RL Training
single_process_model = A2C("MlpPolicy", env_id, verbose=0)

start_time = time.time()
single_process_model.learn(n_timesteps)
total_time_single = time.time() - start_time

print(
    f"Took {total_time_single:.2f}s for single process version - {n_timesteps / total_time_single:.2f} FPS"
)

print(
    "Multiprocessed training is {:.2f}x faster!".format(
        total_time_single / total_time_multi
    )
)

Took 13.94s for multiprocessed version - 1792.78 FPS
Took 47.62s for single process version - 524.96 FPS
Multiprocessed training is 3.42x faster!


In [30]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward:.2f}")

Mean reward: 344.6 +/- 156.10
